In [5]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# pip3 install lxml

# load secrets
load_dotenv()

True

In [30]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [13]:
def get_historical_data(symbol, lookback_days=90):
    iex_key = os.getenv("IEX_API_KEY") 
    historical_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/chart/{lookback_days}d?token={iex_key}'

    historical_data = requests.get(historical_url).json()
    df = pd.DataFrame(historical_data)
    return df[['date', 'high', 'low', 'close']]

# Fetch historical data
ticker = 'PLTR'
historical_df = get_historical_data(ticker)

# Display the historical data
# print(historical_df)

In [14]:
# display html version of df
historical_df.tail(5)

,date,high,low,close
55,2024-01-11,16.93,16.17,16.68
56,2024-01-12,17.06,16.62,16.76
57,2024-01-16,16.74,16.29,16.53
58,2024-01-17,16.41,16.05,16.39
59,2024-01-18,16.81,16.10,16.40


In [27]:
"""Foundational Data for Ichimoku Cloud"""

# Calculate Baseline (Kijun-sen)
historical_df['baseline'] = (historical_df['high'].rolling(window=26).max() + historical_df['low'].rolling(window=26).min()) / 2

# Calculate Turning Line (Tenkan-sen)
historical_df['turning_line'] = (historical_df['high'].rolling(window=9).max() + historical_df['low'].rolling(window=9).min()) / 2

# Calculate Lagging Line (Chikou Span)
historical_df['lagging_line'] = historical_df['close'].shift(26)

# Calculate Leading Span A (Senkou Span A)
historical_df['leading_span_A'] = ((historical_df['baseline'] + historical_df['turning_line']) / 2).shift(-26)

# Calculate Leading Span B (Senkou Span B)
historical_df['leading_span_B'] = (historical_df['high'].rolling(window=52).max() + historical_df['low'].rolling(window=52).min()) / 2
historical_df['leading_span_B'] = historical_df['leading_span_B'].shift(-26)

# Note: we don't drop NA values here because we want to preserve the full dataset for plotting

In [31]:
# print data between rows 27 and 33
historical_df.iloc[27:33] # just shows the in scope data

,date,high,low,close,baseline,turning_line,lagging_line,leading_span_A,leading_span_B
27,2023-11-30,20.23,19.68,20.05,18.165,20.235,16.64,17.3820,18.165
28,2023-12-01,20.29,19.55,20.27,18.165,20.235,15.59,17.2670,18.165
29,2023-12-04,19.90,18.27,18.40,18.165,20.060,15.29,17.0295,18.165
30,2023-12-05,18.55,17.96,18.30,18.165,19.200,15.07,16.8845,18.165
31,2023-12-06,17.83,17.05,17.13,18.165,18.680,14.69,16.8845,18.165
32,2023-12-07,17.48,17.08,17.22,18.165,18.680,14.80,16.8845,18.165
